In [34]:
import torch
import torch.nn as nn
import torchvision.models as models

In [35]:
clanker=models.vgg19(pretrained=True).features

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [36]:
clanker

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [37]:
class Vgg(nn.Module):
  def __init__(self):
    super().__init__()
    self.features=['0','5','10','19','28']
    self.model=models.vgg19(pretrained=True).features[:29]
  def forward(self,x):
    layers=[]
    for no,layer in enumerate(self.model):
      x=layer(x)
      if str(no) in self.features:
        layers.append(x)
    return layers

In [38]:
from PIL import Image

In [39]:
def load(imgn):
  img=Image.open(imgn)
  img=loader(img).unsqueeze(0)
  return img.to(device)

In [40]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
img_size=356

In [42]:
import torchvision.transforms as tfm

In [44]:
loader=tfm.Compose(
    [
        tfm.Resize((img_size,img_size)),
        tfm.ToTensor(),
        tfm.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.255])
    ]
)

In [45]:
org_img=load('/content/WhatsApp Image 2025-09-14 at 00.40.12_649453d2.jpg')
style=load('/content/style7.jpg')
gtd=org_img.clone().requires_grad_(True)

In [46]:
alpha=0.000001
beta=1

In [47]:
optim=torch.optim.Adam([gtd],lr=0.0001)

In [48]:
from torchvision.utils import save_image

In [49]:
def denorm(t):
    mean = torch.tensor([0.485, 0.456, 0.406], device=t.device).view(1,3,1,1)
    std = torch.tensor([0.229, 0.224, 0.225], device=t.device).view(1,3,1,1)
    return t * std + mean

In [52]:
epochs=5001
model=Vgg().to(device).eval()
for epoch in range(epochs):
  gen_feats=model(gtd)
  org_feats=model(org_img)
  style_feats=model(style)

  style_loss=org_loss=0
  for gen_feat,org_feat,style_feat in zip(gen_feats,org_feats,style_feats):
    batch,channel,height,width=gen_feat.shape
    org_loss+=torch.mean((gen_feat-org_feat)**2)
    G=gen_feat.view(channel,height*width).mm(gen_feat.view(channel,height*width).t())
    A=style_feat.view(channel,height*width).mm(style_feat.view(channel,height*width).t())
    style_loss+=torch.mean((G-A)**2)
  tloss=alpha*org_loss+beta*style_loss
  optim.zero_grad()
  tloss.backward()
  optim.step()

  if epoch%200==0:
    print(tloss)
    save_image(denorm(gtd),f'generated{epoch}.png')

tensor(21587956., device='cuda:0', grad_fn=<AddBackward0>)
tensor(20553474., device='cuda:0', grad_fn=<AddBackward0>)
tensor(19582294., device='cuda:0', grad_fn=<AddBackward0>)
tensor(18671558., device='cuda:0', grad_fn=<AddBackward0>)
tensor(17815470., device='cuda:0', grad_fn=<AddBackward0>)
tensor(17011076., device='cuda:0', grad_fn=<AddBackward0>)
tensor(16255877., device='cuda:0', grad_fn=<AddBackward0>)
tensor(15545057., device='cuda:0', grad_fn=<AddBackward0>)
tensor(14873850., device='cuda:0', grad_fn=<AddBackward0>)
tensor(14242292., device='cuda:0', grad_fn=<AddBackward0>)
tensor(13648431., device='cuda:0', grad_fn=<AddBackward0>)
tensor(13087016., device='cuda:0', grad_fn=<AddBackward0>)
tensor(12557490., device='cuda:0', grad_fn=<AddBackward0>)
tensor(12056783., device='cuda:0', grad_fn=<AddBackward0>)
tensor(11583148., device='cuda:0', grad_fn=<AddBackward0>)
tensor(11135796., device='cuda:0', grad_fn=<AddBackward0>)
tensor(10713811., device='cuda:0', grad_fn=<AddBackward0